#### 1. Import libraries

In [ ]:
import math
import numpy
import numba
import pandas
from scipy import special

import torch
from torch import linalg as LA
from torch import digamma, log, sqrt
from torch.special import gammaln, ndtr
torch.set_default_dtype(torch.float64)

from scipy.stats import qmc
from scipy.special import gammaincinv

def Matrix_gradient_gamma(n_row, x_halton, K_p, TAU_p, K, TAU, Mat_dists, weights, Mat_Theta, Mat_Loss_in, Loss_ext, Lambda):
    n_points, n_samples = Mat_dists.shape
    
    BETA = gammaincinv(K, x_halton) / TAU
    THETA_DER_OVER_K = (torch.log(TAU) - torch.digamma(K)).reshape(n_points,1) * Mat_Theta + torch.mean(torch.log(BETA.reshape(n_row, n_points, 1)) * (Mat_dists * torch.ones(n_row, 1, 1) <= BETA.reshape(n_row, n_points, 1)), axis=0)
    THETA_DER_OVER_TAU = (K / TAU).reshape(n_points,1) * (Mat_Theta - special.gdtrc(TAU.reshape(n_points,1).numpy(), (K+1).reshape(n_points,1).numpy(), Mat_dists.numpy()))
    
    Mat_Theta_complement_modified = torch.where((1 - Mat_Theta) == 0, 1e-4, (1 - Mat_Theta))
    Mat_Theta_prod = torch.prod((torch.zeros(n_points, 1, n_samples) + Mat_Theta_complement_modified), axis=1) / Mat_Theta_complement_modified

    vet_K = THETA_DER_OVER_K * (Mat_Loss_in - Mat_Theta_prod * (torch.ones(n_points,1) * Loss_ext))
    vet_TAU = THETA_DER_OVER_TAU * (Mat_Loss_in - Mat_Theta_prod * (torch.ones(n_points,1) * Loss_ext))
    
    weights = weights.reshape(weights.shape[0],)
    
    GRADIENT_K = torch.mean(vet_K * (torch.ones(n_points,1) * weights), axis=1) + (1 / Lambda) * ((K - K_p) * torch.polygamma(1, K) + ((TAU_p - TAU) / TAU))
    GRADIENT_TAU = torch.mean(vet_TAU * (torch.ones(n_points,1) * weights), axis=1) + (1 / Lambda) * ((K_p / TAU) - ((K * TAU_p) / TAU**2))
    return GRADIENT_K, GRADIENT_TAU

#### 2. Mixture Of Linear Experts In Many Regions (Risk Minimization)

##### Binary classification

In [ ]:
def minimize_binary_clf_gn(initial_parameters, X, y, Mat_dists, learning_rate, Lambda, max_iters, weights, print_flag):
    # data shape and number of vicinity points
    n_samples, n_features = X.shape
    n_points = Mat_dists.shape[0]
    
    # get initial_parameters
    gaussian_params, gamma_priors, gamma_params = initial_parameters
    W, MU, SIGMA, w_ext, mu_ext, sigma_ext = gaussian_params
    W.requires_grad_()
    MU.requires_grad_()
    SIGMA.requires_grad_()
    w_ext.requires_grad_()
    mu_ext.requires_grad_()
    sigma_ext.requires_grad_()
    K_p, TAU_p = gamma_priors
    K, TAU = gamma_params

    optimizer = torch.optim.NAdam([{'params': [W, MU, w_ext, mu_ext]},
                                   {'params': [SIGMA, sigma_ext], 'lr': 0.05 * learning_rate},
                                   {'params': [K, TAU], 'lr': 20 * learning_rate}
                                  ], lr=learning_rate)

    terminated_conv, old_risk_bound = 0, 1000
    norm_X_square = LA.norm(X, dim=1)**2
    Max_, max_ = 1000 * torch.ones(n_points), torch.ones(n_points)

    dim, n_row = 1, 60
    sampler = qmc.Halton(dim, scramble=True, seed=0)
    x_halton = sampler.random(n_row)
    
    for i in range(max_iters):
        optimizer.zero_grad()
        
        # compute Mat_Theta
        Mat_Theta = torch.from_numpy(special.gdtrc((TAU.reshape(n_points,1)).numpy(), (K.reshape(n_points, 1)).numpy(), Mat_dists.numpy()))

        # Mat_Loss_in, Loss_ext
        Mat_Loss_in = 1 - ndtr( (y.reshape(n_samples, 1) * (torch.matmul(X, W.T) + MU)) / sqrt(SIGMA**2 + norm_X_square.reshape(n_samples, 1)) ).T
        Loss_ext = 1 - ndtr( (y * (torch.matmul(X, w_ext) + mu_ext)) / sqrt(sigma_ext**2 + norm_X_square) )
        
        # compute KL divergence
        kl = - math.log(torch.prod(SIGMA) * sigma_ext) + 0.5 * (torch.sum(SIGMA**2) + sigma_ext**2 - (n_points + 1))
        kl = kl + 0.5 * ( (LA.norm(W, axis=1)**2).sum() + (MU**2).sum() + LA.norm(w_ext)**2 + mu_ext**2 )
        kl = kl + ( (K - K_p) * digamma(K) + gammaln(K_p) - gammaln(K) + K_p * log(TAU / TAU_p) + K * ((TAU_p - TAU) / TAU) ).sum()

        # compute risk_bound
        loss = torch.sum(Mat_Theta * Mat_Loss_in, axis=0) + torch.prod(1 - Mat_Theta, axis=0) * Loss_ext
        risk_bound = torch.mean(loss * weights) + (1 / Lambda) *  kl
        
        # compute gradient over gaussian parameters
        risk_bound.backward() 
        
        # compute gradient over gamma parameters
        K.grad, TAU.grad = Matrix_gradient_gamma(n_row, x_halton, K_p, TAU_p, K, TAU, Mat_dists, weights, Mat_Theta, Mat_Loss_in, Loss_ext, Lambda)
            
        optimizer.step()                                                              # update gradient
        
        with torch.no_grad():
            K[:] = torch.clamp(K, min=K_p, max=Max_)                                  # contraint : K_p < K < Max_
            TAU[:] = torch.clamp(TAU, min=TAU_p, max=Max_)                            # contraint : TAU_p < TAU < Max_
            
            SIGMA[:] = torch.clamp(torch.abs(SIGMA), max=max_)                        # contraint on 0 < SIGMA < 1
            sigma_ext[:] = torch.clamp(torch.abs(sigma_ext), max=torch.tensor([1.0])) # contraint on 0 < sigma_ext < 1

            # gradients and risk bound stop criterion
            if (i+1)%40 == 0 :
                if ((risk_bound / old_risk_bound) >= 0.95) or ((K / TAU**2).sum() > n_points) :
                    terminated_conv = 1
                    break;
                else:
                    old_risk_bound = risk_bound
                        
    if print_flag:
        if terminated_conv != 0:
            print(f'Converged in {i+1} iteration...')
        else:
            print(f'Terminated in {i+1} iteration... because maximum number of iterations has been exceeded')
            
    return (W.detach().numpy(), MU.detach().numpy(), SIGMA.detach().numpy(), w_ext.detach().numpy(), mu_ext.detach().numpy()[0], sigma_ext.detach().numpy()[0]), (K.numpy(), TAU.numpy())
    

##### Linear regression

In [ ]:
def minimize_reg_gn(initial_parameters, X, y, Mat_dists, learning_rate, Lambda, max_iters, weights, print_flag):
    # data shape and number of vicinity points
    n_samples, n_features = X.shape
    n_points = Mat_dists.shape[0]
    
    # get initial_parameters
    gaussian_params, gamma_priors, gamma_params = initial_parameters
    W, RHO, MU, SIGMA, w_ext, rho_ext, mu_ext, sigma_ext = gaussian_params
    W.requires_grad_()
    RHO.requires_grad_()
    MU.requires_grad_()
    SIGMA.requires_grad_()
    w_ext.requires_grad_()
    rho_ext.requires_grad_()
    mu_ext.requires_grad_()
    sigma_ext.requires_grad_()
    K_p, TAU_p = gamma_priors
    K, TAU = gamma_params

    optimizer = torch.optim.NAdam([{'params': [W, MU, w_ext, mu_ext]},
                                   {'params': [RHO, SIGMA, rho_ext, sigma_ext], 'lr': 0.05 * learning_rate},
                                   {'params': [K, TAU], 'lr': 20 * learning_rate}
                                  ], lr=learning_rate)

    terminated_conv, old_risk_bound = 0, 1000
    norm_X_square = LA.norm(X, dim=1)**2
    Max_, max_ = 1000 * torch.ones(n_points), torch.ones(n_points)

    dim, n_row = 1, 60
    sampler = qmc.Halton(dim, scramble=True, seed=0)
    x_halton = sampler.random(n_row)

    for i in range(max_iters):
        optimizer.zero_grad()
        
        # compute Mat_Theta
        Mat_Theta = torch.from_numpy(special.gdtrc((TAU.reshape(n_points,1)).numpy(), (K.reshape(n_points, 1)).numpy(), Mat_dists.numpy()))
        
        # Mat_Loss_in, Loss_ext
        Mat_Loss_in = (norm_X_square.reshape(n_samples, 1) * RHO**2 + SIGMA**2 + ((torch.matmul(X, W.T) + MU) - y.reshape(n_samples, 1))**2).T
        Loss_ext = norm_X_square * rho_ext**2 + sigma_ext**2 + (torch.matmul(X, w_ext) + mu_ext - y)**2

        # compute KL divergence
        kl = - n_features * math.log(torch.prod(RHO) * rho_ext) - math.log(torch.prod(SIGMA) * sigma_ext) + 0.5 * (n_features * (torch.sum(RHO**2) + rho_ext**2) + (torch.sum(SIGMA**2) + sigma_ext**2) - ((n_features + 1) * n_points + (n_features + 1)))
        kl = kl + 0.5 * ( (LA.norm(W, axis=1)**2).sum() + (MU**2).sum() + LA.norm(w_ext)**2 + mu_ext**2 )
        kl = kl + ( (K - K_p) * digamma(K) + gammaln(K_p) - gammaln(K) + K_p * log(TAU / TAU_p) + K * ((TAU_p - TAU) / TAU) ).sum()
        
        # compute risk_bound
        loss = torch.sum(Mat_Theta * Mat_Loss_in, axis=0) + torch.prod(1 - Mat_Theta, axis=0) * Loss_ext
        risk_bound = torch.mean(loss * weights) + (1 / Lambda) * kl
        
        # compute gradient over gaussian parameters
        risk_bound.backward()  
        
        # compute gradient over gamma parameters
        K.grad, TAU.grad = Matrix_gradient_gamma(n_row, x_halton, K_p, TAU_p, K, TAU, Mat_dists, weights, Mat_Theta, Mat_Loss_in, Loss_ext, Lambda)
            
        optimizer.step()                                                              # update gradient

        with torch.no_grad():
            K[:] = torch.clamp(K, min=K_p, max=Max_)                                  # contraint on K_p < K < Max_
            TAU[:] = torch.clamp(TAU, min=TAU_p, max=Max_)                            # contraint on TAU_p < TAU < Max_

            RHO[:] = torch.clamp(torch.abs(RHO), max=max_)                            # contraint on RHO>0
            SIGMA[:] = torch.clamp(torch.abs(SIGMA), max=max_)                        # contraint on SIGMA>0
            rho_ext[:] = torch.clamp(torch.abs(rho_ext), max=torch.tensor([1.0]))     # contraint on rho_ext>0
            sigma_ext[:] = torch.clamp(torch.abs(sigma_ext), max=torch.tensor([1.0])) # contraint on sigma_ext>0

            # gradients and risk bound stop criterion
            if (i+1)%40 == 0 :
                if ((risk_bound / old_risk_bound) >= 0.95) or ((K / TAU**2).sum() > n_points) :
                    terminated_conv = 1
                    break;
                else:
                    old_risk_bound = risk_bound
                    
    if print_flag:
        if terminated_conv != 0:
            print(f'Converged in {i+1} iteration...')
        else:
            print(f'Terminated in {i+1} iteration... because maximum number of iterations has been exceeded') 

    gaussian_params = (W.detach().numpy(), RHO.detach().numpy(), MU.detach().numpy(), SIGMA.detach().numpy(), w_ext.detach().numpy(), rho_ext.detach().numpy()[0], mu_ext.detach().numpy()[0], sigma_ext.detach().numpy()[0])
    gamma_params = (K.numpy(), TAU.numpy())
    
    return gaussian_params, gamma_params

#### 3. Computation of vicinity

In [ ]:
def num_and_cat_features(dataset, print_var = False):
    category_types_list = []
    colNames = dataset.columns.values.tolist()
    for colName in colNames:
        if dataset[colName].dtypes == 'object' or dataset[colName].dtype.name == 'category':
            category_types_list.append(colName)

    numeric_types_list = pandas.Series(dataset.columns.drop(category_types_list, errors='ignore'))
    if print_var == True:
        print('\n Numeric variables: \n', [i for i in numeric_types_list])
        print('\n Category variables: \n', category_types_list)
    return numeric_types_list, category_types_list

In [ ]:
def get_data_range(df_xtrain=None, data_1=None, data_2=None):
    columns = df_xtrain.columns
    numeric_vars, category_vars = num_and_cat_features(df_xtrain, print_var = False)
    xtrain_range = pandas.DataFrame(columns = columns, dtype = float)
    xtrain_range.loc[0,numeric_vars] = numpy.ptp(df_xtrain[numeric_vars], axis=0)
    xtrain_range[category_vars] = 1
    
    if data_1 is None:
        return xtrain_range.values[0]
    elif data_2 is None:
        data_1_range = pandas.DataFrame(columns = columns, dtype = float)
        data_1_range.loc[0,numeric_vars] = numpy.ptp(data_1[numeric_vars], axis=0)
        data_1_range[category_vars] = 1
        return xtrain_range.values[0], data_1_range.values[0]
    else:
        data_1_range = pandas.DataFrame(columns = columns, dtype = float)
        data_1_range.loc[0,numeric_vars] = numpy.ptp(data_1[numeric_vars], axis=0)
        data_1_range[category_vars] = 1
        
        data_2_range = pandas.DataFrame(columns = columns, dtype = float)
        data_2_range.loc[0,numeric_vars] = numpy.ptp(data_2[numeric_vars], axis=0)
        data_2_range[category_vars] = 1
        return xtrain_range.values[0], data_1_range.values[0], data_2_range.values[0]

def distance_processing(df_x0=None, df_xtrain=None, data_1=None, data_2=None):
    numeric_vars, category_vars = num_and_cat_features(df_xtrain, print_var = False)
    for col in category_vars:
        df_xtrain.loc[:, col] = (df_xtrain[col] == df_x0[col].values[0]).values.astype(int)
    
    if data_1 is None:
        df_x0.loc[:, category_vars] = 1
        return df_x0.values[0], df_xtrain.values
    elif data_2 is None:
        for col in category_vars:
            data_1.loc[:, col] = (data_1[col] == df_x0[col].values[0]).values.astype(int)
        df_x0.loc[:, category_vars] = 1
        return df_x0.values[0], df_xtrain.values, data_1.values
    else:
        for col in category_vars:
            data_1.loc[:, col] = (data_1[col] == df_x0[col].values[0]).values.astype(int)
            data_2.loc[:, col] = (data_2[col] == df_x0[col].values[0]).values.astype(int)
        df_x0.loc[:, category_vars] = 1
        return df_x0.values[0], df_xtrain.values, data_1.values, data_2.values

In [ ]:
@numba.jit(nopython=True, fastmath=True)
def fast_euclidean_dist(x_1d, y_1d):
    dist = [(a - b)**2 for a, b in list(zip(x_1d, y_1d))]
    dist = numpy.array(dist)
    return numpy.sqrt(numpy.sum(dist, axis=0))

@numba.jit(nopython=True, fastmath=True)
def fast_gower_dist(x_1d, y_1d, z_1d):
    dist = [numpy.abs(a - b) for a, b in list(zip(x_1d, y_1d))]
    dist = numpy.array(dist) / z_1d
    return numpy.sum(dist, axis=0) / len(x_1d)

@numba.jit(nopython=True, fastmath=True)
def compute_distance(dist_type='gower', x0=None, X=None, X_range=None):
    distance = numpy.zeros(X.shape[0])
    
    if dist_type=='gower': 
        for i in numba.prange(X.shape[0]):
            distance[i] = fast_gower_dist(x0, X[i], X_range)
    elif dist_type=='euclidean':
        for i in numba.prange(X.shape[0]):
            distance[i] = fast_euclidean_dist(x0, X[i])
    return distance

In [ ]:
def vicinity(dist_type='gower', df_x0=None, df_xtrain=None, df_xtest_1=None, df_xtest_2=None):
    
    if df_xtest_1 is None:
        xtrain_range = get_data_range(df_xtrain=df_xtrain, data_1=None, data_2=None)
        x0, xtrain = distance_processing(df_x0=df_x0, df_xtrain=df_xtrain, data_1=None, data_2=None)
        return compute_distance(dist_type=dist_type, x0=x0, X=xtrain, X_range=xtrain_range)
    
    elif df_xtest_2 is None:
        xtrain_range = get_data_range(df_xtrain=df_xtrain, data_1=None, data_2=None)
        x0, xtrain, xtest = distance_processing(df_x0=df_x0, df_xtrain=df_xtrain, data_1=df_xtest_1, data_2=None)
        dist_train = compute_distance(dist_type=dist_type, x0=x0, X=xtrain, X_range=xtrain_range)
        dist_test = compute_distance(dist_type=dist_type, x0=x0, X=xtest, X_range=xtrain_range)
        return dist_train, dist_test
    
    else:
        xtrain_range = get_data_range(df_xtrain=df_xtrain, data_1=None, data_2=None)
        x0, xtrain, xtest_1, xtest_2 = distance_processing(df_x0=df_x0, df_xtrain=df_xtrain, data_1=df_xtest_1, data_2=df_xtest_2)
        dist_train = compute_distance(dist_type=dist_type, x0=x0, X=xtrain, X_range=xtrain_range)
        dist_test_1 = compute_distance(dist_type=dist_type, x0=x0, X=xtest_1, X_range=xtrain_range)
        dist_test_2 = compute_distance(dist_type=dist_type, x0=x0, X=xtest_2, X_range=xtrain_range)
        return dist_train, dist_test_1, dist_test_2

def set_of_vicinity(dist_type='gower', df_X0=None, df_xtrain=None, df_xtest_1=None, df_xtest_2=None):
    DIST_TRAIN, DIST_TEST_1, DIST_TEST_2 = [], [], []
    idx = df_X0.index
    
    if df_xtest_1 is None:
        for i in range(len(idx)):
            DIST_TRAIN.append(vicinity(dist_type=dist_type, df_x0=df_X0.loc[idx[i]:idx[i]], df_xtrain=df_xtrain))
        return numpy.array(DIST_TRAIN)
    
    elif df_xtest_2 is None:
        for i in range(len(idx)):
            dist_train, dist_test = vicinity(dist_type=dist_type, df_x0=df_X0.loc[idx[i]:idx[i]], df_xtrain=df_xtrain, df_xtest_1=df_xtest_1)
            DIST_TRAIN.append(dist_train)
            DIST_TEST_1.append(dist_test)
        return numpy.array(DIST_TRAIN), numpy.array(DIST_TEST_1)
            
    else:
        for i in range(len(idx)):
            dist_train, dist_test_1, dist_test_2 = vicinity(dist_type=dist_type, df_x0=df_X0.loc[idx[i]:idx[i]], df_xtrain=df_xtrain, df_xtest_1=df_xtest_1, df_xtest_2=df_xtest_2)
            DIST_TRAIN.append(dist_train)
            DIST_TEST_1.append(dist_test_1)
            DIST_TEST_2.append(dist_test_2)
        return numpy.array(DIST_TRAIN), numpy.array(DIST_TEST_1), numpy.array(DIST_TEST_2)